In [2]:
!pip install pandas
!pip install openpyxl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Vorwort

Diese Projektarbeit wurde von erstellt von

- 
- 

Die Einzelnen bzw. gemeinsamen Leistungen sind durch Kürzel hinter den einzelnen Überschriften kenntlich gemacht. 
- (E) | 
- (F) | 
- (EF)| gemeinsame Arbeit

Dieses Notebook ist ebenfalls, wie in der E-Mail angekündigt, mit Auswertung nach vollständigen Validierungsläufen, auf GitHub verfügbar, wurde dort jedoch hinsichtlich persönlicher Daten bearbeitet.

### Externe Links

Der Code und die Modelle sind hier auf Github verfügbar:

https://github.com/f-schreiber/judement_classification.git


Die genutzten Daten sind hier auf Sciebo verfügbar:

https://fh-swf.sciebo.de/s/Bgdsk7TDZFP6Pw8

# Projektübersicht (EF)

In der Vergangenheit war die Nutzung der meisten Machine Learning basierten Natural Language Processing Technologien in der Anwendung auf einen spezifischen Sachverhalt immer mit dem erstellen von umfangreichen Trainings- Test- und Validierungsdatensätzen verbunden. Zusätzlich war der Anwendungsbereich der entwickelten Modelle of sehr spezifisch. Das hat sich mit der Entwicklung von Transformer Modellen und den daraus entstandenen Anwendungen wie zum Beispiel ChatGPT drastisch verändert. Es ist möglich geworden, ein vortrainiertes Modell für eine Vielzahl von verschiedenen Anwendungsfällen zu nutzen. 

Ziel dieses Projektes ist es zu evaluieren, ob LLMs dazu in der Lage sind, Fragen zu Jurisitschen Sachverhalten zu beantworten. Dazu wird den Sprachmodellen ein Gerichtsurteil zur Verfügung gestellt, für welches es eine Frage zu beantworten gilt. Um die generelle Machbarkeit zu testen, wird in diesem Projekt mit vergleichsweise einfachen Fragen gearbeitet. 

Dazu werden die folgenden Punkte im Rahmen dieses Projektes umgesetzt:
- Es werden zwei Pipelines zur zur Klassifikation umgesetzt.
- Es wird wird die Möglichkeit geschaffen, ein Finetuning eines Sprachmodelles auf Gerichtsurteilen vorzunehmen
- Die von Sprachmodellen generierten Ergebnisse werden mit händisch anotierten Daten verglichen.

### Herausforderungen

Herausforderungen welche in diesem Projekt erwartet werden:
- Modelle welche lokal auf dem Cluster genutzt werden können sind in ihrer Paramateranzahl und damit Fähigkeiten begrenzt im Vergleich zu z.B. OpenAI Modellen 
- Gerichtsurteile, Fragen und Prompts sind in deutscher Sprache verfasst. Die Trainingsdaten von LLMs sind jedoch primär englisch.


# Verwandte Literatur

## Pretrained Classifier (E)

LLMs für Klassifizierungsentscheidungen einzusetzen ist etwas das bereits an anderen Stellen untersucht wurde[1]. Das passierte auch schon sehr kurz, nachdem ChatGPT veröffentlicht wurde. Damals allerdings mit Ergebnissen, welche schlechter waren als aufgabenspezifische Modelle. Auch Zero-shot-Classification [2] ist etwas, was relativ früh nachdem LLMs populär wurden untersucht wurde. Ebenfalls untersucht wurde, ob und wie mehrere Modelle eine gemeinsame Entscheidung treffen können[3].

## Finetuning  (F)

Das Finetuning von LLMs für Klassifikationsentscheidungen im juristischen Bereich ist etwas, das in der Vergangenheit erfolgreich erprobt wurde [4]. Auch das Finetuning mittels der LoRA Methode im juristischen Kontext ist etwas [5], das bereits in der Vergangenheit untersucht worden ist. Dies auch an verschiedenen Stellen, auch außerhalb der englischen Sprache [6].










[1] (https://arxiv.org/abs/2305.08377)

[2] (https://www.sciencedirect.com/science/article/pii/S2095809922006324)

[3] (https://arxiv.org/pdf/2307.12973)

[4] (https://www.sciencedirect.com/science/article/abs/pii/S0267364923000742

[5] (https://link.springer.com/chapter/10.1007/978-981-97-5569-1_17)

[6] (https://ceur-ws.org/Vol-3877/paper7.pdf)


# Daten

## Datenbeschaffung (E)
Für das Projekt wurden Gerichtsurteile von der Website www.rechtsprechung-im-internet.de, einer Seite vom Bundesministerium der Justiz, gescrapt. Die genutzten Daten sind vom Stand Mitte Dezember, die danach veröffentlichten Urteile wurden nicht mehr berücksichtigt.
Die Urteile stammen von den folgenden Gerichtshöfen: 

- Bundesverfassungsgericht
- Bundesgerichtshof
- Bundesverwaltungsgericht
- Bundesfinanzhof
- Bundesarbeitsgericht
- Bundessozialgericht
- Bundespatentgericht
- Gemeinsamer Senats der obersten Gerichtshöfe des Bundes

Insgesamt umfasst der Datensatz 75500 Entscheidungen.
Um die Dateien herunterzuladen, wurden zuerst die Links zu den Urteilen extrahiert. Die Dateien wurden danach automatisch heruntergeladen. Um die Server zu entlasten und eine automatisierte Traffic-Blockierung zu umgehen und so das Scrapen aller Dateien zu ermöglichen, wurde ein Download-Delay mit Zufallskomponente eingebaut. Die XML-Dateien wurden mit Beautifulsoup geparst. Der Textinhalt wurde extrahiert und in txt-Dateien abgespeichert. Diese wurden in Ordnern entsprechend des erstellenden Gerichts abgelegt.

## Datensätze (F)

Im Rahmen diese Projektes werden zwei Datensätze benötigt. Einer dient der Validierung der am Ende erstellten Klassifikationspipelines, der andere dient dem Domänenspezifischen Finetuning einer LLMs.
### Validierung

Der Datensatz zur Validierung der Pipeline enthält 100 zufällig ausgewählte Gerichtsurteile. Diese wurden in zwei Arbeitspakete von je 50 Gerichtsurteilen unterteilt, jedes der Teammitglieder labelte anschließend eines der Arbeitspakete. Die Arbeitspakete wurden hinsichtlich der folgenden vier Fragen gelabelt:
- Werden in dem Urteil Zahlungsstreitigkeiten behandelt?
- Behandelt das Urteil den Arbeitsschutz?
- Wird eine Stafttat gegen die körperliche Unversehrheit betrachtet?
- Ist der Streitgegenstand ein Patentsachverhalt?

Wenn die Fragen mit "Ja" beantwortet werden konnten wurde eine "1" eingetragen, sonst eine "0". Dieser Datensatz ist in seinen Klassen nicht ausbalanciert, was bei den Metriken für eine spätere Pipelinebewertung Berücksichtigung finden muss.

Die Aufteilung des Datensatzes ist dabei die Folgende:

| Frage | 1  | 2  | 3  | 4  |
|-------|----|----|----|----|
| ja    | 52 | 3  | 4  | 8  |
| nein  | 48 | 97 | 96 | 92 |
|       |    |    |    |    |

Es ist erkennbar, das es lediglich für Frage 1 einen balancierten Datensatz gibt. Für Fragen 2 und 3 ist es fraglich ob sich bei dieser Stichprobengröße überhaupt gesicherte aussagen getroffen werden können. 

Der Code für die Erzeugung eines ungelabelten Datensatzes findet sich im Notebook Validation_preprocessing.ipynb

### Finetuning

Der Datensatz zum Finetuning des Modells wurde ebenfalls aus zufällig gewählten Gerichtsurteilen aus dem Gesamtkorpus erstellt. Hier wurde sichergestellt, dass der Finetuning Datensatz nicht durch in der Validierung genutzten Gerichtsurteile kontaminiert wurde. Dafür wurden die Validierungsurteile beim erstellen des Validierungsdatensatzes in entsprechenden Ordner verschoben. Diese Änderungen wurden nicht im auf Sciebo verfügbaren Datensatz vorgenommen.  Die Grundmenge der Daten hat einen Umfang von circa 447 Millionen Token. Da bei Gerichtsurteilen der obersten deutschen Gerichtshöfe eine konsistent hohe Sprachqualität angenommen werden kann, werden die Trainingsdaten vor einem Trainingsdurchlauf zufällig ausgewählt, und keine designierten Trainingsdatensätze erstellt. 

In [1]:
import os
from transformers import AutoTokenizer
from tqdm import tqdm

model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

folder_path = "Gerichtsurteile"

def count_tokens_in_txt_files(folder_path):
    total_tokens = 0
    txt_files = []
    
    for root, _, files in os.walk(folder_path):
        txt_files.extend(os.path.join(root, f) for f in files if f.endswith(".txt"))
    
    for file_path in tqdm(txt_files, desc="Processing files"):
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()
            tokens = tokenizer.encode(text, truncation=False)
            total_tokens += len(tokens)

    print(f"Total tokens: {total_tokens}")

count_tokens_in_txt_files(folder_path)


Processing files: 100%|██████████| 75462/75462 [12:54<00:00, 97.42it/s] 

Total tokens: 446658748


# Evaluierungspipeline (E)


## Genutzte Technik
 
### Summarization Pipeline
Zuerst wurde eine Summarization Pipeline implementiert, die auf den Beispielen aus der Vorlesung basiert. Als erstes Modell wurde das Modell "Einmalumdiewelt/T5-Base_GNAD" von HuggingFace gewählt, da dies auf die deutsche Sprache trainiert wurde. Das Modell wurde erfolgreich integriert und generierte eine Zusammenfassung. Allerdings konnte es nur maximal 512 Token als Input verarbeiten, was für die meisten Gerichtsurteile nicht ausreichend ist. Daher wurde nach einem anderen Modell gesucht.

Als zweites Modell wurde das Modell "mistralai/Mistral-7B-Instruct-v0.2" integriert und getestet, da es sich hierbei auch um ein Modell handelt was auf die deutsche Sprache trainiert wurde, allerdings auch eine Texteingabe von bis zu 32.000 Token verarbeiten kann, womit es sich gut zum Zusammenfassen der Gerichtsurteile eignet. Das Modell wurde in die Summarization Pipeline integriert, da dieses Modell allerdings nicht explizit für das Zusammenfassen von Texten generiert wurde, funktionierte dies nicht. Deswegen wurde die Pipeline zu einer Generated Text Pipeline geändert mit der Idee, den Text anhand eines Prompts zusammenzufassen. Das Mistral-7B-Modell wurde erfolgreich in die Generated Text Pipeline integriert und lieferte auch bei längeren Inputs eine Zusammenfassung, allerdings traten auch hier bei sehr langen Texten vereinzelt Probleme auf, worauf später eingegangen wird.

### Text Generation Pipeline

Um die Leistung der Pipeline eventuell zu steigern, wurde noch ein weiteres Modell zum Vergleich herangezogen. Dafür wurde das etwas aktuellere Modell von Mistral AI das "mistralai/Mistral-Small-24B-Instruct-2501" in die Pipeline integriert. Das Modell benötigt zwar mehr Rechenkapazität, was durch die Hochschulserver aber kein Problem darstellt und liefert längere Outputs. Zudem weist das Modell durch seine 24 Milliarden Parametern ein besseres Sprachverständnis auf. Das Modell weist präzise Zusammenfassungen und eine signifikant bessere Prompt Adherence auf, die entscheidend für die Klassifizierung ist. Neben den bereits erwähnten Modellen wurden außerdem die Modelle "lora-results_2m" und "lora-results_4m" getestet.

## Prompt Engineering

Um das qualitativ beste Ergebnis aus der Pipeline herauszuholen, wurden verschiedene Prompts ausprobiert. Da alle Gerichtsurteile auf deutsch sind, wurden ausschließlich deutsche Prompts verwendet. Trotzdem gab das Modell bei den Antworten nicht nur “Ja” und “Nein”, sondern auch Antworten auf Englisch zurück, weshalb zusätzlich auf “yes” und “no” geprüft wurde. Da es beim Prompt Engineering besonders wichtig ist, den Prompt möglichst klar und präzise zu gestalten, wurde explizit darauf hingewiesen, eine Antwort mit Ja oder Nein zu generieren. Zusätzlich wurde die entsprechende Frage und der zusammengefasste Text bzw. Abschnitt mit in den Prompt gegeben.

## Erstellte Validierungspipelines

Die evaluation_with_summary-Pipeline generiert zuerst eine Zusammenfassung des Textes, wenn die Zusammenfassung weiterhin über 4000 Token liegt, wird der Text erneut zusammengefasst. Das wird weitere zwei Male wiederholt, falls die generierte Zusammenfassung weiterhin über 4000 Token bleibt. Ist diese Grenze auch im letzten Durchgang überschritten, wird später eine “3” zurückgegeben. Wenn der Text erfolgreich zusammengefasst wird, werden die generierten Antworten anschließend ausgewertet, indem überprüft wird, ob die Antworten die Wörter “ja” oder “nein” enthalten. Da einige Antworten auf Englisch verfasst wurden, wurde ebenfalls auf “yes” und “no” geprüft. Da diese Wörter meist am Anfang oder Ende der Antwort stehen, werden nur die jeweils ersten und letzten fünf Buchstaben überprüft. Dadurch wird das Risiko minimiert, dass Wörter, die „ja“, „nein“, „yes“ oder „no“ enthalten, fälschlicherweise als Antwort erkannt werden.
Nach der Prüfung des jeweiligen Textes wird eine “1” für ein “ja” und eine “0” für ein “nein” zurückgegeben. Falls keines von beidem enthalten ist, wird eine “2” ausgegeben 

Die evaluation_no_summary-Pipeline überprüft zuerst, ob die Textlänge die 4000 Token überschreitet. Ist dies der Fall, wird der Text in mehrere Abschnitte mit maximal 2000 Token unterteilt, die jeweils zusammengefasst werden. Auch hier werden wieder nur die ersten fünf beziehungsweise die letzten fünf Buchstaben überprüft. Falls ein Abschnitt mit “1” oder “0” klassifiziert wird, wird der gesamte Text entsprechend eingeordnet. Kommt keines der Wörter “ja”,”nein”,”yes” oder “no” in den Abschnitten vor, wird für den Text “2” ausgegeben.

# Finetuning (F)

Im Rahmen des Projektes wurde ein Finetuning an einem pretrained Transformer Model vorgenommen. Dabei wurden zwei Möglichkeiten untersucht, ein Finetuning für ein solches generatives Sprachmodel vorzunehmen.


### Quantisiertes Finetuning 

Beim Quantisierten Finetuning werden die Gewichte des Modells nicht als full precision float Zahlen (Datentyp fp32) sondern als Zahlen eines anderen Datentypes geladen, welcher weniger Speicherbits benötigen. Dabei gibt es unterschiedliche Quantisierungsgrade, abhängig davon, wie stark die Anzahl der genutzten Bits pro Gewicht verringert werden. Mittels dieser Quantisierung wird der Speicherbedarf während des Trainings und der Inferenz stark reduziert. Außerdem wird die Anzahl an $\frac{Token}{s}$ welche von Model erzeugt werden können erhöht.

Die Anwendung dieser Methode wurde im Rahmen dieses Projektes untersucht, aus technischen Gründen jedoch nicht durchgeführt. Das Problem war, dass die aktuellen kompatiblen Versionen der Transformers Bibliothek und der bitsandbytes Bibliothek, innerhalb von BitsandBytes eine Abhängigkeit zum trition Paket in einer Version >= 3.0.0 haben. Diese Version benötigt allerdings eine neuere CUDA Version als Version 11.7 welche auf dem Cluster der Hochschule installiert ist. 

### Low Rank Adaption (LoRA)

Bei der Low Rank Adaption wird nicht das vortrainierte Model selbst trainiert, sondern es werden die für die Gewichtsmatrizen eines Teil des Models zwei kleinere Matrizen approximiert, welche anschließend trainiert werden, während der Rest des Modells eingefroren und nicht trainiert wird. Dadurch wird der benötigte Speicherplatz während des Trainings massiv reduziert, da die Anzahl der trainierten Parameter nur einen Burchteil der Gesamtparameter eines Modelles betragen. 

Durch die Reduktion der zu trainierenden Parameter wird auch der Bedarf an benötigten Trainingsdaten stark reduziert, was diese Methode besonders attraktiv für das Finetuning von Domänenspezifischen Modellen macht, wie in diesem Projekt umgesetzt.  

Das Finetuning wurde für ein Mistral-7b Instruct Model durchgeführt. Es wurde ebenfalls getestet, ein Finetuning für ein 24B Mistral Model vorzunehmen. Hierfür war jedoch der GPU-Speicher auf einer einzelnen Grafikkarte des Clusters nicht ausreichend, selbst wenn das Training auf einer anderweitig freien Grafikkarte durchgeführt wurde. Multi-GPU Training wäre ein Möglichkeit, auch ein solches Model zu trainieren, dies geht jedoch mit zusätzlichem technischen Aufwand einher, für welchen im aktuellen Projekt keine Kapazitäten vorhanden sind. 

### Umsetzung 
Die Umsetzung des Finetunings erfolgte im zweiten Teil des LoRA_Finetuning.ipynb Notebooks. 

Unter Berückschtigung dessen, dass sowohl LoRA Methode als auch die technische Infrastruktur nicht auf die vorhandenen Datenmengen ausgelegt sind, wurden zwei Finetunes durchgeführt, einer mit zwei Millionen Token über drei Epochen. Ein zweiter mit vier Millionen Tokens über vier Epochen. Die Trainingszeiten lagen hier bei einer circa Stunde für den ersten Datensatz, und bei circa 2 Stunden und 45  Minuten für den zweiten Datensatz.

# Auswertung (EF)

## Beobachtungen während der Pipeline Ausführung

- Mit dem 27b Modell ist die zusammenfassende Pipeline schneller als die nicht zusammenfassende
- Bei den 7b Modellen ist die nicht zusammenfassende Pipeline schneller als die zusammenfassende
- Der Durchlauf einer Pipeline mit einem Model dauert ohne anderweitige Nutzung der Grafikkarte welche zur Ausführung genutzt wird ca. 3-5 Stunden

### Metrik

Als Metrik wurde der Macro-F1 Score genutzt. Dieser wurde gewählt, da er auch in der Lage dazu ist, mit nicht balancierten Klassen zu arbeiten. Die Abwägungen hinsichtlich dessen, welche Fehlerwie schwer wiegen, ist etwas wozu wir aufgrund von fehlender Domänenexpertiese nicht in der Lage sind. Deshalb wurden aller Fehler als gleich schwer angenommen. 


## Ergebnisse

Für die Pipeline, in welcher keine Zusammenassung genutzt wurde wurden im ersten Ausschnitt der Validierungsdaten die folgenden Ergebnisse erzielt:

In [41]:
import pandas as pd
from sklearn.metrics import f1_score
import os


reference_data = pd.read_excel("Sample_combined.xlsx")
reference_data = reference_data.iloc[:, 2:]
results_path = "no_summary_results"
model_results = []

for file in os.listdir(results_path):
    if file.endswith(".csv") and "b-instruct" in file:  # Process only CSV files
        file_path = os.path.join(results_path, file)
        large_sum_data = pd.read_csv(file_path).iloc[:, 1:]
        large_sum_data.columns = reference_data.columns
        
        temp_df = pd.DataFrame()
        
        for col in reference_data.columns:
            data_subset = pd.DataFrame({0: reference_data[col], 1: large_sum_data[col]})
            data_subset_filtered = data_subset[data_subset[1].isin([0, 1])].copy()
            data_subset.replace([2, 3], 0, inplace=True)
            f1_raw = f1_score(data_subset[0], data_subset[1], average="macro")
            f1_filter = f1_score(data_subset_filtered[0], data_subset_filtered[1], average="macro")
            temp_df[col] = [f1_raw, f1_filter, data_subset_filtered.shape[0]]
        
        model_results.append(temp_df.copy())


                        

    


In [42]:
no_sum_frame = pd.concat(model_results)
no_sum_frame.index = ["Macro F1 nicht gefilter - 7b","Makro F1 gefilter - 7b","Anzahl korrekt formatierte Antworten",
                      "Macro F1 nicht gefilter - 24b","Makro F1 gefilter - 24b","Anzahl korrekt formatierte Antworten"]
no_sum_frame

,Werden in dem Urteil Zahlungsstreitigkeiten behandelt?,Behandelt das Urteil den Arbeitsschutz?,Wird eine Stafttat gegen die körperliche Unversehrheit betrachtet?,Ist der Streitgegenstand ein Patentsachverhalt?
Macro F1 nicht gefilter - 7b,0.698916,0.376299,0.345489,0.485893
Makro F1 gefilter - 7b,0.667755,0.338361,0.259707,0.420202
Anzahl korrekt formatierte Antworten,74.000000,84.000000,78.000000,77.000000
Macro F1 nicht gefilter - 24b,0.469277,0.487179,0.621212,0.680140
Makro F1 gefilter - 24b,0.478934,0.488764,0.617284,0.677193
Anzahl korrekt formatierte Antworten,92.000000,91.000000,93.000000,92.000000


Im Vergleich kann deutlich erkannt werden, dass die Prompt Adherance des 24b Models signifikant besser ist, als die des 7b Modelles. Das 7b Modell hat in circa 78 Prozent der Fälle entsprechend des vorgegebene Schemas geantworte, des 24b Modell in 92 Prozent der Fälle. Während es für das 24b Modell kaum unterschiede zwischen dem gefilterten und ungefilterem Makro F1 Score gibt, sind diese Unterschiede beim 7b Modell deutlich.


Das 7b Modell performt für Zahlungsstreitigkeiten signifikant besser als das 24b Modell, in allen anderen Klassen jedoch signifikant schlechter. Insgesamt ist der durchschnittliche Makro F1 Score des 24b Modelles deutlich höher als der des 7b Modelles. Dies ist konsistent mit den zu erwartenden Ergebnissen.

Für die Pipeline, in welcher für zu lange Gerichtsurteile eine Zusammenfassung erstellt wurde wurden die folgenden Ergebnisse erzielt:

In [33]:
os.listdir(results_path)[1:2]

['24b-instruct_summary.csv']

In [57]:
reference_data = pd.read_excel("Sample_combined.xlsx")
reference_data = reference_data.iloc[:, 2:]
results_path = "summary_results"
model_results = []

a = os.listdir(results_path)
a.sort(reverse=True)

for file in a:
    if file.endswith(".csv") and "b-instruct" in file:  # Process only CSV files
        file_path = os.path.join(results_path, file)
        large_sum_data = pd.read_csv(file_path).iloc[:, 1:]
        large_sum_data.columns = reference_data.columns
        
        temp_df = pd.DataFrame()
        
        for col in reference_data.columns:
            data_subset = pd.DataFrame({0: reference_data[col], 1: large_sum_data[col]})
            data_subset_filtered = data_subset[data_subset[1].isin([0, 1])].copy()
            data_subset.replace([2, 3], 0, inplace=True)
            f1_raw = f1_score(data_subset[0], data_subset[1], average="macro")
            f1_filter = f1_score(data_subset_filtered[0], data_subset_filtered[1], average="macro")
            temp_df[col] = [f1_raw, f1_filter, data_subset_filtered.shape[0]]
        
        model_results.append(temp_df.copy())


In [58]:
sum_frame = pd.concat(model_results)
sum_frame.index = [#"Macro F1 nicht gefilter - 2mft","Makro F1 gefilter - 2mft","Anzahl korrekt formatierte Antworten"
                   "Macro F1 nicht gefilter - 7b","Makro F1 gefilter - 7b","Anzahl korrekt formatierte Antworten",
                   "Macro F1 nicht gefilter - 24b","Makro F1 gefilter - 24b","Anzahl korrekt formatierte Antworten"]
sum_frame

,Werden in dem Urteil Zahlungsstreitigkeiten behandelt?,Behandelt das Urteil den Arbeitsschutz?,Wird eine Stafttat gegen die körperliche Unversehrheit betrachtet?,Ist der Streitgegenstand ein Patentsachverhalt?
Macro F1 nicht gefilter - 7b,0.441595,0.470899,0.555556,0.524457
Makro F1 gefilter - 7b,0.662150,0.452381,0.478145,0.537500
Anzahl korrekt formatierte Antworten,34.000000,46.000000,39.000000,37.000000
Macro F1 nicht gefilter - 24b,0.379960,0.492386,0.739583,0.473684
Makro F1 gefilter - 24b,0.436275,1.000000,0.774923,0.477273
Anzahl korrekt formatierte Antworten,69.000000,71.000000,73.000000,69.000000


Im vergleich der beiden beiden Pipelines lässt sich klar feststellen, dass die Prompt Adherance in der Zusammenfassenden Pipeline klar gegenüber der nicht zusammenfassenden Pipeline nachlässt. Auch die Streuung der F1-Scores ist bei der zusammenfassenden Pipeline größer als bei der nicht zusammenfassenden Pipeline. 

Die Performance des 7b Modelles, ist tendenziell besser, als in der nicht zusammenfassenden Pipeline. Die des 24b Modelles ist tendenziell schlechter als in der nicht zusammenfassenden Pipeline. 

Sollten weitere Experimente mit größeren Modellen stattfinden, so ist es zu empfehlen, diese mit der nicht zusammenfassenden Pipeline durchzuführen.

## Fehlschlag des Finetunings (F)

Das LoRA Finetuning des 7b-Mistral Modelles ist in der Anwendung in beiden Pipelines fehlgeschlagen, da keine Prompt Adherance mehr erreicht wurde. Es wird vermutet, dass ein overfitting auf den Trainingsdaten erfolgt ist. Entsprechend ist es empfehlenswert, das Training noch einmal grundlegend zu hinterfragen. Auch, ob es sinnvoll ist, ein 7b-Modell zu trainieren, für größere Modelle müsste jedoch ein multi-GPU training erfolgen.

# Future Research

### Technische Umsetzung (F)

Um eine Verbesserung des Laufzeitverhaltens, insbesondere auch die Nutzung mehrerer parallel laufender GPUs zu erreichen sollte es in Betracht gezogen werden die technische Umsetzung nicht mehr mittels pipelines aus dem transformers package zu realisieren. Stattdessen könnte es in betracht gezogen werden, externe Tools wie zum Beispiel ollama [7]. Dies würde auch eine Integration mit höher abstrahierten Frameworks wie z.B. Langchain deutlich vereinfachen, da nicht mehr das Model direkt angesprochen werden muss, sondern eine API, ähnlich wie bei On-Demand Anbietern wie z.B. OpenAI oder AWSBedrock. Weiter sollten generell mehr Modelle auch verschiedener

### Prompt Engineering (E)
Im Prompt Engineering sollten noch weitere Anpassungen vorgenommen werden, z.B. sollte statistisch abgesichert getest werden wie sich die Ergebnisse verändern, wenn noch einmal größere Änderungen am Prompt duchgeführt werden. Zum Beispiel könnte der Prompt auf englisch formuliert werden. Das könnte, aufgrund des hohen englischen Sprachanateils in den Trianingsdaten die Performance noch einmal verändern. Vorerst sollte jedoch der Fokus darauf gelegt werden, die Durchlaufzeit der Validierungspipeline zu verbessern.

### Finetuning (F)

Um das Finetuning und dessen Qualität zukünftig noch zu verbessern, sollten weitere Tests hinsichtlich Größe der Trainingsdaten, der Anzahl der Epochen, sowie der Matrixgröße durchgeführt werden. Um diese Training jedoch effektiv validieren zu können muss zuerst eine Performanceverbesserung innerhalb der Validierungspipeline erreicht werden.




[7] (https://ollama.com/)